In [ ]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO)

# from folium import LayerControl
import xarray as xr
import matplotlib.pyplot as plt

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

In [ ]:
# load a geometry
ids = [
 "S1A_IW_SLC__1SDV_20230904T063730_20230904T063757_050174_0609E3_DAA1", 
 "S1A_IW_SLC__1SDV_20230916T063730_20230916T063757_050349_060FCD_6814"
]

In [ ]:
data_dir = "/data/S1"
primary_dir = f"{data_dir}/{ids[0]}.SAFE"
secondary_dir = f"{data_dir}/{ids[1]}.SAFE"
out_dir = f"/data/res/Morocco-earthquake-2023"

In [ ]:
from eo_tools.S1.process import slc2geo, coherence, amplitude
file_prm = f"{out_dir}/primary.tif"
file_sec = f"{out_dir}/secondary.tif"
file_coh = f"{out_dir}/coh.tif"
file_coh_dask = f"{out_dir}/coh_dask.tif"
# computing amplitude and complex coherence  in the radar geometry
# coherence(file_prm, file_sec, file_coh, box_size=[3,10], magnitude=False)


In [ ]:
def coh_dask(file_prm, file_sec, file_out, box_size=5, magnitude=True):
    """Compute the complex coherence from two SLC image files.

    Args:
        file_prm (str): GeoTiff file of the primary SLC image
        file_sec (str): GeoTiff file of the secondary SLC image
        file_out (str): output file
        box_size (int, optional): Window size in pixels for boxcar filtering. Defaults to 5.
        magnitude (bool, optional): Writes magnitude only. Otherwise a complex valued raster is written. Defaults to True.
    """
    import dask.array as da
    import numpy as np
    import xarray as xr
    from rasterio.errors import NotGeoreferencedWarning
    from eo_tools.S1.util import boxcar
    import warnings

    log.info("Computing coherence")

    if isinstance(box_size, list):
        box_az = box_size[0]
        box_rg = box_size[1]
    else:
        box_az = box_size
        box_rg = box_size

    ds_prm = xr.open_dataset(file_prm, lock=False, chunks="auto", engine="rasterio")
    ds_sec = xr.open_dataset(file_sec, lock=False, chunks="auto", engine="rasterio")

    prm = ds_prm["band_data"][0].data
    sec = ds_sec["band_data"][0].data

    process_args = dict(dimaz=box_az, dimrg=box_rg, dtype="float32", depth=(box_rg, box_az))

    coh = da.map_overlap(boxcar, prm * sec.conj(), **process_args)
    coh /= np.sqrt(da.map_overlap(boxcar, prm * prm.conj(), **process_args)).real
    coh /= np.sqrt(da.map_overlap(boxcar, sec * sec.conj(), **process_args)).real

    if magnitude:
        coh = np.abs(coh)
        nodataval = np.nan
    else:
        nodataval = np.nan + 1j * np.nan

    da_coh = xr.DataArray(
        data=coh[None],
        dims=("band", "y", "x"),
    )
    da_coh.rio.write_nodata(nodataval)

    warnings.filterwarnings("ignore", category=NotGeoreferencedWarning)
    da_coh.rio.to_raster(file_out)
    # return coh


coh_dask(file_prm, file_sec, file_coh_dask, box_size=[3, 10], magnitude=True)

ds_coh = xr.open_dataset(file_coh_dask)
coh = ds_coh["band_data"][0]

plt.figure(figsize=(10, 10))
plt.imshow(coh[::2,::8], interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)